In [8]:
import sqlite3
import pandas as pd
from sqlite3 import Error
from matplotlib import pyplot as plt

In [2]:
def create_connection(db_file):    
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print("connexion établie")
        return conn
    except Error as e:
        print(e)

    return conn

In [3]:
with open('orders.sql', 'r') as sql_file:
    sql_script = sql_file.read()

In [5]:
conn = sqlite3.connect('../data/marketplaces.db')
cur = conn.cursor()
cur.executescript(sql_script)

OperationalError: no such column: order_status

In [31]:
# get rounded coordinates of each zip codes
cur.execute('''
            SELECT geolocation_zip_code_prefix, AVG(geolocation_lat), AVG(geolocation_lng)
            FROM geolocations
            GROUP BY geolocation_zip_code_prefix
            ''')
coordinates_df = pd.DataFrame(cur.fetchall(), columns = ['Zip', 'Latitude', 'Longitude'])

# get the rest
cur.execute('''
            SELECT DISTINCT
            freight_value,
            (JULIANDAY(order_delivered_customer_date)
            - JULIANDAY(order_delivered_carrier_date)), seller_zip_code_prefix, customer_zip_code_prefix
            FROM items i
            INNER JOIN orders o ON i.order_id = o.order_id
            INNER JOIN sellers s ON s.seller_id = i.seller_id
            INNER JOIN customers c ON c.customer_id = o.customer_id
            ''')
shipping_df = pd.DataFrame(cur.fetchall(), columns = ['Freight Value', 'Shipping Duration', 'Seller Zip', 'Customer Zip'])

# merge info with proper zip coordinates
shipping_df = pd.merge(shipping_df, coordinates_df, how='left', left_on='Seller Zip', right_on='Zip')
shipping_df = pd.merge(shipping_df, coordinates_df, how='left', left_on='Customer Zip', right_on='Zip')
shipping_df = shipping_df.drop(columns=['Zip_x', 'Zip_y'])
shipping_df = shipping_df.rename(columns={
                                          'Latitude_x':'Seller Latitude',
                                          'Longitude_x':'Seller Longitude',
                                          'Latitude_y':'Customer Latitude',
                                          'Longitude_y':'Customer Longitude'
                                         })
print(shipping_df)

       Freight Value  Shipping Duration  Seller Zip  Customer Zip  \
0              13.29           1.214954       27277         28013   
1              19.93           8.062083        3471         15775   
2              17.87           6.029491       37564         35661   
3              12.79           4.003229       14403         12952   
4              18.14          13.289144       87900         13226   
...              ...                ...         ...           ...   
98899          43.41          15.449769       88303         65077   
98900          36.53           6.518692        1206         81690   
98901          16.95           1.881343       80610          4039   
98902           8.72           1.122766        4733         13289   
98903          12.79           3.156551       14940         18605   

       Seller Latitude  Seller Longitude  Customer Latitude  \
0           -22.497188        -44.127324         -21.763186   
1           -23.565754        -46.519097     

In [17]:
cur.execute('''
            SELECT DISTINCT
            freight_value, product_weight_g, product_length_cm, product_height_cm, product_width_cm
            FROM items INNER JOIN products ON items.product_id = products.product_id
            ''')
dimension_df = pd.DataFrame(cur.fetchall(), columns = ['Freight Value', 'Product Weight (g)', 'Product Length (cm)', 'Product Height (cm)', 'Product Width (cm)'])
print(dimension_df)

       Freight Value  Product Weight (g)  Product Length (cm)  \
0              13.29               650.0                 28.0   
1              19.93             30000.0                 50.0   
2              17.87              3050.0                 33.0   
3              12.79               200.0                 16.0   
4              18.14              3750.0                 35.0   
...              ...                 ...                  ...   
68315          44.02              2700.0                 60.0   
68316          10.96               100.0                 60.0   
68317          22.06               200.0                 25.0   
68318          43.41             10150.0                 89.0   
68319          36.53              8950.0                 45.0   

       Product Height (cm)  Product Width (cm)  
0                      9.0                14.0  
1                     30.0                40.0  
2                     13.0                33.0  
3                     1